In [47]:
from os import environ

environ['PYTHONDONTWRITEBYTECODE'] = '1'

In [48]:
TRAINING_SET_PORCENTAGE: float = 0.73
WINDOW_PERIODS: int = 14

NAME: str = 'Tesla Stocks Uncertainty Simple Perceptron HFT'
DESCRIPTION: str = 'Uncertainty Simple Perceptron Model, for High-Frequency Trading for Tesla (risk 3:1 and 14 periods of 15 minutes).'
AUTHOR: str = 'Dylan Sutton Chavez'

EPOCHS: int = 170
PATIENCE: int = 20
LEARNING_RATE: float = 0.0001

SYMBOL: str = 'TSLA'

# RETRAIN_THRESHOLD: int = 17

In [49]:
from dotenv import load_dotenv
from os import getenv

load_dotenv()

ALPACA_KEY: str = getenv('ALPACA_KEY')
ALPACA_SECRET: str = getenv('ALPACA_SECRET')

from apis.alpaca_markets import AlpacaMarkets

alpaca_markets = AlpacaMarkets(ALPACA_KEY, ALPACA_SECRET, SYMBOL)

In [50]:
historical_market_bars: dict[str, any] = alpaca_markets.historical_market_bars(limit_bars=None, weeks_data_window=144)[SYMBOL]
length_historical_market: int = len(historical_market_bars)

training_set_max_index = round((length_historical_market) * TRAINING_SET_PORCENTAGE)

from database.duck_db import DuckDB

training_db = DuckDB(f'{SYMBOL.lower()}_training.set')
training_db.truncate()
training_set = [bar for bar in historical_market_bars[:training_set_max_index]]

test_db = DuckDB(f'{SYMBOL.lower()}_test.set')
test_db.truncate()

config_db = DuckDB(f'{SYMBOL.lower()}_config.set')
config_db.truncate()

training_volume_set = [bar.volume for bar in training_set]
training_trade_count_set = [bar.trade_count for bar in training_set]
training_vwap_sep = [bar.vwap for bar in training_set]

In [51]:
from feature_encoders.z_score import ZScore

volume_zscore_obj: ZScore = ZScore(training_volume_set)
trade_count_zscore_obj: ZScore = ZScore(training_trade_count_set)
vwap_zscore_obj: ZScore = ZScore(training_vwap_sep)

zscore_config = {
    'volume_means': volume_zscore_obj.means,
    'volume_std': volume_zscore_obj.std,
    
    'trade_count_means': trade_count_zscore_obj.means,
    'trade_count_std': trade_count_zscore_obj.std,

    'vwap_means': vwap_zscore_obj.means,
    'vwap_std': vwap_zscore_obj.std
}

config_db.insert(zscore_config)

In [52]:
from datetime import datetime

from features_vectorizer import TimeSeriesConfig, TimeConfig, NormalizationConfig, features_vectorizer

training_set_max_index_offset: int = training_set_max_index - 1
training_set = []

for index, bar in enumerate(historical_market_bars):
    raw_window: list[dict] = historical_market_bars[index: index + WINDOW_PERIODS]
    raw_prices_window: list[float] = [bar_dict.close for bar_dict in raw_window]

    if len(raw_prices_window) < WINDOW_PERIODS:
        break

    price_window_range: float = bar.high - bar.low

    stop_loss: float = 0.5 * price_window_range
    take_profit: float = 1.5 * price_window_range

    bar_timestamp: datetime = bar.timestamp
    bar_formated_timestamp: str = bar_timestamp.strftime('%Y-%m-%d %H:%M %Z')

    bar_volume = bar.volume
    bar_trade_count = bar.trade_count
    bar_vwap = bar.vwap

    time_series_config: TimeSeriesConfig = TimeSeriesConfig(raw_vector=raw_prices_window, periods=WINDOW_PERIODS)
    time_config: TimeConfig = TimeConfig(current_minute=bar_timestamp.minute + 1, current_hour=bar_timestamp.hour + 1, current_day_of_week=bar_timestamp.weekday() + 1, current_month=bar_timestamp.month)
    normalization_config: NormalizationConfig = NormalizationConfig(volume_zscore_obj=volume_zscore_obj, volume=bar_volume, trade_count_zscore_obj=trade_count_zscore_obj, trade_count=bar_trade_count, vwap_zscore_obj=vwap_zscore_obj, vwap=bar_vwap)

    features_vector = features_vectorizer(time_series_config, time_config, normalization_config)

    entry_bar_price = bar.close

    up_take_profit_price = entry_bar_price + take_profit
    down_take_profit_price = entry_bar_price - take_profit

    next_price_bars = historical_market_bars[index + 1: index + 1 + WINDOW_PERIODS]
    bar_label = 0.5

    for next_bar in next_price_bars:
        if next_bar.high >= up_take_profit_price:
            bar_label = 1
            break

        if next_bar.low <= down_take_profit_price:
            bar_label = 0
            break

    bar_metadata_vectorized = {
        'timestamp': bar_formated_timestamp,
        'vector': features_vector,
        'label': bar_label
    }

    if index <= training_set_max_index_offset:
        training_set.append(bar_metadata_vectorized)

    if index >= training_set_max_index_offset:
        test_db.insert(bar_metadata_vectorized)

In [55]:
training_set_sanitized = []

for example in training_set:
    if example['label'] == 0.5:
        continue

    training_set_sanitized.append(example)

training_db.truncate_and_insert_list(training_set_sanitized)

In [72]:
from core.uncertainty_simple_perceptron import UncertaintySimplePerceptron

uncertainty_simple_perceptron = UncertaintySimplePerceptron(NAME, DESCRIPTION, AUTHOR)

uncertainty_simple_perceptron.train(training_db.database_file_name, EPOCHS, PATIENCE, LEARNING_RATE)

Current Epoch: 1/170 | Error Rate: 0.4938587702697688 | Elapsed Time (ms): 63.0097
Current Epoch: 2/170 | Error Rate: 0.483914059890905 | Elapsed Time (ms): 64.9019
Current Epoch: 3/170 | Error Rate: 0.4709265649931352 | Elapsed Time (ms): 61.8516
Current Epoch: 4/170 | Error Rate: 0.45081450146573154 | Elapsed Time (ms): 63.0429
Current Epoch: 5/170 | Error Rate: 0.4352666147166871 | Elapsed Time (ms): 63.7244
Current Epoch: 6/170 | Error Rate: 0.4361571857953913 | Elapsed Time (ms): 60.622
Current Epoch: 7/170 | Error Rate: 0.4388660061597833 | Elapsed Time (ms): 62.5773
Current Epoch: 8/170 | Error Rate: 0.4437270399643772 | Elapsed Time (ms): 64.2902
Current Epoch: 9/170 | Error Rate: 0.4465100745853278 | Elapsed Time (ms): 66.4891
Current Epoch: 10/170 | Error Rate: 0.4490704664366025 | Elapsed Time (ms): 63.3526
Current Epoch: 11/170 | Error Rate: 0.447066681509518 | Elapsed Time (ms): 62.5674
Current Epoch: 12/170 | Error Rate: 0.43894022041634195 | Elapsed Time (ms): 63.2723
Cu

In [73]:
from numpy import array

successful_pred: int = 0
failed_pred: int = 0
uncertainty_pred: int = 0

for example in test_db.line_by_line():
    inference_prediction = uncertainty_simple_perceptron.inference(input_features=array(example['vector']), epsilon=0.0017)

    prediction, net_output = inference_prediction

    if prediction == example['label']:
        successful_pred += 1

    elif prediction == 0.5:
        uncertainty_pred += 1

    else:
        failed_pred += 1

print(successful_pred, failed_pred, uncertainty_pred)

2316 56 9537
